# NYC Taxi Rides

In this notebook we show some more advanced features that are useful for building composed views, a.k.a., dashboards.

In [1]:
%load_ext autoreload
%autoreload 2

### Download the data

In [42]:
import pandas as pd
from utils import download_file

nyc_data = pd.read_csv('data/nyc-taxi.csv')

for data in nyc_data.iterrows():
    download_file(*data[1])

File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`


### Create tilesets

In [46]:
from higlass.tilesets import cooler

dropoff = {
    'count': cooler('data/nyctaxi-dropoff.count.1m.mcool', name='Total Dropoffs'),
}
pickup = {
    'count': cooler('data/nyctaxi-pickup.count.1m.mcool', name='Total Pickups'),
}

### Visualize pickups and drop-offs

In [4]:
from higlass import display
from higlass.client import Track, View

nyc_x=[11789206, 11827005]
nyc_y=[15044734, 15081957]

In [5]:
view_config = {
    'width': 6,
    'initialXDomain': nyc_x,
    'initialYDomain': nyc_y,
}

pickups_view = View(
    x=0,
    tracks=[Track('heatmap', tileset=pickup['count'], height=400)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track('heatmap', tileset=dropoff['count'], height=400)],
    **view_config
)

widget, server, viewconf = display([pickups_view, dropoffs_view])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'NcYdYV40Qnu9a…

### Synchronize view location and zoom level

In [6]:
widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'NcYdYV40Qnu9a…

### Add crosshairs and tooltips

In [7]:
## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
    }
}
##############################

pickups_view = View(
    x=0,
    tracks=[Track(tileset=pickup['count'], **track_config)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track(tileset=dropoff['count'], **track_config)],
    **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'NUywEkqcQjyvk…

### Overview+details

In [41]:
from copy import deepcopy

## Important change ##########
view_config_details = deepcopy(view_config)
view_config_details['y'] = 6
view_config_details['initialXDomain'] = [11820321, 11826194]
view_config_details['initialYDomain'] = [15073235, 15079019]

pickups_view_details = View(
    x=0,
    tracks=[Track(tileset=pickup['count'], **track_config)],
    **view_config_details
)

dropoffs_view_details = View(
    x=6,
    tracks=[Track(tileset=dropoff['count'], **track_config)],
    **view_config_details
)

pickups_view_overview = View(
    x=0,
    tracks=[
        Track(tileset=pickup['count'], **track_config),
        Track('viewport-projection-center', fromViewUid=pickups_view_details.uid),
    ],
    **view_config
)

dropoffs_view_overview = View(
    x=6,
    tracks=[
        Track(tileset=dropoff['count'], **track_config),
        Track('viewport-projection-center', fromViewUid=dropoffs_view_details.uid),
    ],
    **view_config
)
##############################

widget, server, viewconf = display(
    views=[
        pickups_view_overview, dropoffs_view_overview,
        pickups_view_details, dropoffs_view_details
    ],
    location_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
    zoom_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'GhBpbpJdRm2Ib…

### Compare different attributes through data operations

In [28]:
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True
    }
}

pickups_track = Track(tileset=pickup['count'], **track_config)
dropoffs_track = Track(tileset=dropoff['count'], **track_config)

pickups_view = View(
    x=0, tracks=[pickups_track], **view_config
)

dropoffs_view = View(
    x=6, tracks=[dropoffs_track], **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'RIyZIThBT_uGC…

Let's divide pickups by dropoff by directly adjusting the view config

In [35]:
from higlass.client import ViewConf
from higlass.viewer import HiGlassDisplay

diverging_colormap = [
    '#0085cc',
    '#62cef6',
    '#eeeeee',
    '#ff66b8',
    '#bf0066',
]

diff_track = (
    pickups_track
        .change_options(
            name='Pickups divided by dropoffs',
            colorRange=diverging_colormap)
        .change_attributes(
            server=server.api_address,
            # With this magic setup we tell HiGlass to divide pickups by dropoffs
            data={
                'type': 'divided',
                'children': [{
                    'server': server.api_address,
                    'tilesetUid': pickup['count'].uuid
                }, {
                    'server': server.api_address,
                    'tilesetUid': dropoff['count'].uuid
                }]
            }
        )
)

diff_view_config = deepcopy(view_config)
diff_view_config['width'] = 12
diff_view = View(tracks=[diff_track], **diff_view_config)

# Finally, we need to create a new view config and pass that into the widget
HiGlassDisplay(viewconf=ViewConf([diff_view]).to_dict())

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'RaCRuA5yTB-Qje_vnEORIQ', 'tracks': {'top': [], '…

- **Pink** indicates areas with more *pickups*
- **Blue** indicates areas with more *dropoffs*

### Superimpose tracks

In [38]:
view_config_together = deepcopy(view_config)
view_config_together['width'] = 12

## Important change ##########
heatmap_track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'backgroundColor': 'transparent',
        'colorRange': [
            'rgba(255, 255, 255, 0.05)',
            'rgba(245, 166,  35, 0.4)',
            'rgba(208,   2,  27, 0.85)',
            'black',
        ],
    }
}
mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqZXB2aWd4NDBmZTIzM3BjdGZudTFob2oifQ.Jnmp1xWJyS4_lRhzrZAFBQ',
        'style': 'light-v9',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
    }
}
##############################

pickups_only = View(
    tracks=[
        Track(**mapbox_track_config),
        Track(tileset=dropoff['count'], **heatmap_track_config),
    ],
    **view_config_together
)

widget, server, viewconf = display([pickups_only])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'MiiJ-qqPRQOGE…

### Finally... let's turn the lights off

In [40]:
heatmap_track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'backgroundColor': 'transparent',
        ## Important change ##########
        'colorRange': [
            'rgba(0, 0, 0, 0.01)',
            'rgba(34, 46, 84, 0.25)',
            'rgba(68, 141, 178, 0.5)',
            'rgba(104, 191, 48, 0.7)',
            'rgba(253, 255, 84, 0.8)',
            'rgba(253, 255, 255, 0.9)',
        ],
        ##############################
        'colorbarBackgroundColor': 'black',
        'colorbarBackgroundOpacity': 0.5,
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    }
}
mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqZXB2aWd4NDBmZTIzM3BjdGZudTFob2oifQ.Jnmp1xWJyS4_lRhzrZAFBQ',
        'style': 'dark-v8',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    }
}

pickups_in_dark = View(
    tracks=[
        Track(**mapbox_track_config),
        Track(tileset=dropoff['count'], **heatmap_track_config),
    ],
    **view_config_together
)

## Important change ##########
widget, server, viewconf = display([pickups_in_dark], dark_mode=True)
##############################

widget

HiGlassDisplay(hg_options={'isDarkTheme': True}, viewconf={'editable': True, 'views': [{'uid': 'RJw2rliKStylwP…

---
---
---

## Other cool NYC properties worth looking at and comparing

In [44]:
import pandas as pd
from utils import download_file

nyc_data = pd.read_csv('data/nyc-taxi-extended.csv')

for data in nyc_data.iterrows():
    download_file(*data[1])

File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`


In [45]:
from higlass.tilesets import cooler

dropoff = {
    'count': cooler('data/nyctaxi-dropoff.count.1m.mcool', name='Total Dropoffs'),
    'passenger_count': cooler('data/nyctaxi-dropoff.passenger_count.mean.1m.mcool', name='Dropoffs Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-dropoff.tip_amount.mean.1m.mcool', name='Dropoffs Mean Tip'),
    'total_amount': cooler('data/nyctaxi-dropoff.total_amount.mean.1m.mcool', name='Dropoffs Mean Total Amount'),
    'trip_distance': cooler('data/nyctaxi-dropoff.trip_distance.mean.1m.mcool', name='Dropoffs Mean Trip Distance'),
}
pickup = {
    'count': cooler('data/nyctaxi-pickup.count.1m.mcool', name='Total Pickups'),
    'passenger_count': cooler('data/nyctaxi-pickup.passenger_count.mean.1m.mcool', name='Pickups Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-pickup.tip_amount.mean.1m.mcool', name='Pickups Mean Tip'),
    'total_amount': cooler('data/nyctaxi-pickup.total_amount.mean.1m.mcool', name='Pickups Mean Total Amount'),
    'trip_distance': cooler('data/nyctaxi-pickup.trip_distance.mean.1m.mcool', name='Pickups Mean Trip Distance'),
}

In [36]:
## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
    }
}
##############################

pickup_amount_track = Track(tileset=pickup['total_amount'], **track_config)
pickup_dist_track = Track(tileset=pickup['trip_distance'], **track_config)
dropoff_amount_track = Track(tileset=dropoff['total_amount'], **track_config)
dropoff_dist_track = Track(tileset=dropoff['trip_distance'], **track_config)

pickup_amount_view = View(x=0, y=0, tracks=[pickup_amount_track], **view_config)
pickup_dist_view = View(x=0, y=6, tracks=[pickup_dist_track], **view_config)
dropoff_amount_view = View(x=6, y=0, tracks=[dropoff_amount_track], **view_config)
dropoff_dist_view = View(x=6, y=6, tracks=[dropoff_dist_track], **view_config)

widget, server, viewconf = display(
    views=[
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    ],
    location_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
    zoom_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'KwtWjwVbRveW7…

In [37]:
from higlass.client import ViewConf
from higlass.viewer import HiGlassDisplay

## Important change ##########
pickup_div_track = (
    pickup_amount_track
        .change_options(
            name='Pickups: Amount by Distance')
        .change_attributes(
            server=server.api_address,
            # With this magic setup we tell HiGlass to divide
            # total amount by the trip distance
            data={
                'type': 'divided',
                'children': [{
                    'server': server.api_address,
                    'tilesetUid': pickup['total_amount'].uuid
                }, {
                    'server': server.api_address,
                    'tilesetUid': pickup['trip_distance'].uuid
                }]
            }
        )
)
##############################

dropoff_div_track = (
    dropoff_amount_track
        .change_options(
            name='Dropoffs: Amount by Distance')
        .change_attributes(
            server=server.api_address,
            # With this magic setup we tell HiGlass to divide
            # total amount by the trip distance
            data={
                'type': 'divided',
                'children': [{
                    'server': server.api_address,
                    'tilesetUid': dropoff['total_amount'].uuid
                }, {
                    'server': server.api_address,
                    'tilesetUid': dropoff['trip_distance'].uuid
                }]
            }
        )
)


pickup_amount_by_dist_view = View(
    x=0, y=0, tracks=[pickup_div_track], **view_config
)

dropoff_amount_by_dist_view = View(
    x=6, y=0, tracks=[dropoff_div_track], **view_config
)

## Important change ##########
div_viewconf = ViewConf([
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
],
location_syncs=[(
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
)],
zoom_syncs=[(
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
)])


HiGlassDisplay(viewconf=div_viewconf.to_dict())
##############################

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'I-iKW2jQTE2NdiaGEabnAA', 'tracks': {'top': [], '…